In [1]:
from math import exp
from math import log10
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size // 2) ** 2 / float(2 * sigma ** 2)) for x in range(window_size)])
    return gauss / gauss.sum()

In [3]:
def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    #.mm - Performs a matrix multiplication of the matrice
    
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window

In [4]:
def _ssim(img1, img2, window, window_size, channel, size_average=True):
    mu1 = F.conv2d(img1, window, padding=window_size // 2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size // 2, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size // 2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size // 2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size // 2, groups=channel) - mu1_mu2

    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

In [5]:
def ssim(img1, img2, window_size=110, size_average=True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)

    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)

    return _ssim(img1, img2, window, window_size, channel, size_average)

In [6]:
mse_metrics = 0.0
rmse95_metrics = 0.0
rmse_wind_metrics = 0.0
rmse_slp_metrics = 0.0
ssim_metrics = 0.0
psnr_metrics = 0.0

In [8]:
field = np.load('data/HiRes/Arrays/FIELD/field_2009.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2009.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

In [9]:
msk = np.load('data/HiRes/Arrays/Percentile/Mask_95.npy')
mask = np.zeros_like(wind_real)

for i in range(0,472):
    mask[i,:,:] = msk[0,:,:]
for i in range(472,1208):
    mask[i,:,:] = msk[1,:,:]
for i in range(1208,1944):
    mask[i,:,:] = msk[2,:,:]
for i in range(1944,2672):
    mask[i,:,:] = msk[3,:,:]
for i in range(2672,2920):
    mask[i,:,:] = msk[0,:,:]

In [10]:
field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics))

In [11]:
rmse_wind_metrics

1.4186985766988789

In [12]:
rmse95_metrics

1.837051195803751

In [13]:
rmse_slp_metrics

0.573412258339384

In [14]:
psnr_metrics

35.25768516499361

In [15]:
field = np.load('data/HiRes/Arrays/FIELD/field_2010.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2010.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 2))

In [ ]:
#ssim_year = ssim(torch.from_numpy(field_interp_norm), torch.from_numpy(field_norm)).item()      
#ssim_metrics += ssim_year

In [16]:
rmse_wind_metrics/2

1.4290101152241677

In [17]:
rmse95_metrics/2

1.8804529347907661

In [18]:
rmse_slp_metrics/2

0.5693507058639113

In [20]:
psnr_metrics

35.42111420803743

In [21]:
field = np.load('data/HiRes/Arrays/FIELD/field_2011.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2011.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 3))

In [22]:
rmse_wind_metrics/3

1.4295488027383882

In [23]:
rmse95_metrics/3

1.880225783093158

In [24]:
rmse_slp_metrics/3

0.5694470457432342

In [25]:
psnr_metrics

34.72823381223792

In [26]:
field = np.load('data/HiRes/Arrays/FIELD/field_2012.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2012.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

msk = np.load('data/HiRes/Arrays/Percentile/Mask_95.npy')
mask = np.zeros_like(wind_real)

for i in range(0,480):
    mask[i,:,:] = msk[0,:,:]
for i in range(480,1216):
    mask[i,:,:] = msk[1,:,:]
for i in range(1216,1952):
    mask[i,:,:] = msk[2,:,:]
for i in range(1952,2680):
    mask[i,:,:] = msk[3,:,:]
for i in range(2680,2928):
    mask[i,:,:] = msk[0,:,:]

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 4))

In [27]:
rmse_wind_metrics/4

1.4302844339480751

In [28]:
rmse95_metrics/4

1.8858839921282629

In [29]:
rmse_slp_metrics/4

0.571511768773811

In [30]:
psnr_metrics

35.43503372713172

In [31]:
field = np.load('data/HiRes/Arrays/FIELD/field_2013.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2013.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

msk = np.load('data/HiRes/Arrays/Percentile/Mask_95.npy')
mask = np.zeros_like(wind_real)

for i in range(0,472):
    mask[i,:,:] = msk[0,:,:]
for i in range(472,1208):
    mask[i,:,:] = msk[1,:,:]
for i in range(1208,1944):
    mask[i,:,:] = msk[2,:,:]
for i in range(1944,2672):
    mask[i,:,:] = msk[3,:,:]
for i in range(2672,2920):
    mask[i,:,:] = msk[0,:,:]

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 5))

In [32]:
rmse_wind_metrics/5

1.4319145895662395

In [33]:
rmse95_metrics/5

1.8846636972256992

In [34]:
rmse_slp_metrics/5

0.5724475210521721

In [35]:
psnr_metrics

35.47967770411597

In [36]:
field = np.load('data/HiRes/Arrays/FIELD/field_2014.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2014.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

msk = np.load('data/HiRes/Arrays/Percentile/Mask_95.npy')
mask = np.zeros_like(wind_real)

for i in range(0,472):
    mask[i,:,:] = msk[0,:,:]
for i in range(472,1208):
    mask[i,:,:] = msk[1,:,:]
for i in range(1208,1944):
    mask[i,:,:] = msk[2,:,:]
for i in range(1944,2672):
    mask[i,:,:] = msk[3,:,:]
for i in range(2672,2920):
    mask[i,:,:] = msk[0,:,:]

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 6))

In [37]:
rmse_wind_metrics/6

1.4336984043950818

In [38]:
rmse95_metrics/6

1.8950045969029483

In [39]:
rmse_slp_metrics/6

0.57384619961254

In [40]:
psnr_metrics

35.02858298383442

In [41]:
field = np.load('data/HiRes/Arrays/FIELD/field_2015.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2015.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

msk = np.load('data/HiRes/Arrays/Percentile/Mask_95.npy')
mask = np.zeros_like(wind_real)

for i in range(0,472):
    mask[i,:,:] = msk[0,:,:]
for i in range(472,1208):
    mask[i,:,:] = msk[1,:,:]
for i in range(1208,1944):
    mask[i,:,:] = msk[2,:,:]
for i in range(1944,2672):
    mask[i,:,:] = msk[3,:,:]
for i in range(2672,2920):
    mask[i,:,:] = msk[0,:,:]

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 7))

In [42]:
rmse_wind_metrics/7

1.4361767888728154

In [43]:
rmse95_metrics/7

1.897363669812486

In [44]:
rmse_slp_metrics/7

0.576601691249192

In [45]:
psnr_metrics

35.07976155095143

In [47]:
field = np.load('data/HiRes/Arrays/FIELD/field_2016.npy')
field_interp = np.load('data/Interp/FIELD/field_cubic_2016.npy')

wind_real = np.sqrt(np.square(field[:,0,:,:]) + np.square(field[:,1,:,:]))
wind_interp = np.sqrt(np.square(field_interp[:,0,:,:]) + np.square(field_interp[:,1,:,:]))

msk = np.load('data/HiRes/Arrays/Percentile/Mask_95.npy')
mask = np.zeros_like(wind_real)

for i in range(0,480):
    mask[i,:,:] = msk[0,:,:]
for i in range(480,1216):
    mask[i,:,:] = msk[1,:,:]
for i in range(1216,1952):
    mask[i,:,:] = msk[2,:,:]
for i in range(1952,2680):
    mask[i,:,:] = msk[3,:,:]
for i in range(2680,2927):
    mask[i,:,:] = msk[0,:,:]

field_norm = np.zeros_like(field)
field_interp_norm = np.zeros_like(field_interp)

field_norm[:,0,:,:] = (field[:,0,:,:]+40)/80
field_norm[:,1,:,:] = (field[:,1,:,:]+40)/80
field_norm[:,2,:,:] = (field[:,2,:,:]-940)/120

field_interp_norm[:,0,:,:] = (field_interp[:,0,:,:]+40)/80
field_interp_norm[:,1,:,:] = (field_interp[:,1,:,:]+40)/80
field_interp_norm[:,2,:,:] = (field_interp[:,2,:,:]-940)/120

diff_msk = (wind_real - mask) > 0
wind_real_msk = wind_real * diff_msk
wind_interp_msk = wind_interp * diff_msk

mse_year = np.mean(np.square(field_norm - field_interp_norm))
rmse_wind_year = np.sqrt(np.mean(np.square(wind_real - wind_interp)))
rmse95_year = np.sqrt(np.sum(np.square(wind_real_msk - wind_interp_msk))/np.sum(diff_msk))
rmse_slp_year = np.sqrt(np.mean(np.square(field[:,2,:,:] - field_interp[:,2,:,:])))

mse_metrics += mse_year
rmse95_metrics += rmse95_year
rmse_wind_metrics += rmse_wind_year
rmse_slp_metrics += rmse_slp_year

psnr_metrics = 10 * log10(np.square(np.max(field_interp_norm)) / (mse_metrics / 8))

In [48]:
rmse_wind_metrics/8

1.4356595539108943

In [49]:
rmse95_metrics/8

1.8976389783668184

In [50]:
rmse_slp_metrics/8

0.5762417411895537

In [51]:
psnr_metrics

35.15908203751223